In [13]:
import numpy
import pandas
import tensorflow as tf
from datetime import datetime
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

print(datetime.now())

images_train = pandas.read_csv('./images_train.csv').values
images_validate = pandas.read_csv('./images_validate.csv').values
images_test = pandas.read_csv('./images_test.csv').values
labels_train = pandas.read_csv('./labels_train.csv').values
labels_validate = pandas.read_csv('./labels_validate.csv').values
labels_test = pandas.read_csv('./labels_test.csv').values

images_train = images_train.astype(numpy.float)
images_validate = images_validate.astype(numpy.float)
images_test = images_test.astype(numpy.float)

images_train = numpy.multiply(images_train, 1.0 / 255.0)
images_validate = numpy.multiply(images_validate, 1.0 / 255.0)
images_test = numpy.multiply(images_test, 1.0 / 255.0)

labels_train = labels_train.ravel()
labels_validate = labels_validate.ravel()
labels_test = labels_test.ravel()

print("length of training: ", len(labels_train))
print("length of validation", len(labels_validate))
print("length of testing", len(labels_test))
print("length of training: ", len(images_train))
print("length of validation", len(images_validate))
print("length of testing", len(images_test))

2017-04-09 19:08:15.426439
length of training:  28708
length of validation 3588
length of testing 3588
length of training:  28708
length of validation 3588
length of testing 3588


In [14]:
sess = tf.InteractiveSession()

def one_hot(label, labels_unique):
    index_offset = numpy.arange(label.shape[0]) * labels_unique
    one_hot = numpy.zeros((label.shape[0], labels_unique))
    one_hot.flat[index_offset + label.ravel()] = 1
    labelset = one_hot.astype(numpy.uint8)  
    return labelset

labels_unique = 7
labels_trainset = one_hot(labels_train, labels_unique)
labels_validateset = one_hot(labels_validate, labels_unique)
labels_testset = one_hot(labels_test, labels_unique)

images_train = images_train.reshape([-1, 48, 48, 1])
images_validate = images_validate.reshape([-1, 48, 48, 1])
images_test = images_test.reshape([-1, 48, 48, 1])

print("end")

end


In [ ]:
network = input_data(shape=[None, 48, 48, 1])
network = conv_2d(network, 64, 5, activation='relu')
network = local_response_normalization(network)
network = max_pool_2d(network, 3, strides=2)
network = conv_2d(network, 64, 5, activation='relu')
network = max_pool_2d(network, 3, strides=2)
network = conv_2d(network, 128, 4, activation='relu')
network = dropout(network, 0.3)
network = fully_connected(network, 3072, activation='relu')
network = fully_connected(network, 7, activation='softmax')
network = regression(network,
                                  optimizer='momentum',
                                  loss='categorical_crossentropy')

print("end")

end


In [ ]:
model = tflearn.DNN(network, checkpoint_path='emotion_recognition',
                    max_checkpoints=1, tensorboard_verbose=2)    
model.fit(
            images_train, labels_trainset,
            n_epoch=1,
            batch_size=50,
            shuffle=True,
            show_metric=True,
            snapshot_step=200,
            snapshot_epoch=True,
            run_id='emotion_recognition'
        )

In [ ]:
score = model.evaluate(images_train, labels_train)
print('validation accuarcy: %0.4f%%' % (score[0] * 100))

score = model.evaluate(images_validate, labels_validate)
print('validation accuarcy: %0.4f%%' % (score[0] * 100))

score = model.evaluate(images_test, labels_test)
print('Test accuarcy: %0.4f%%' % (score[0] * 100))